# Results and Observables

*What you will learn:*

- how to configure an `Observable` to measure quantities of interest in an emulation;
- what observables are available by default;
- how to retrieve the measured observables from a `Results` instance.

## The `Observable` mechanism

As showcased in the page on [backend execution](/tutorials/backends.nblink#4.-Configuring-the-Backend), the `Observable` mechanism provides an efficient and uniform way of calculating and storing different quantities of interest throughout an **emulation**. 

It is a shared mechanism between the different emulator backends that can generally be used interchangeably with minimal to no modifications.

### Available Observables

<div class="alert alert-info">

Info

The observables below are typical quantities of interest that can generally be given to any emulator backend. 

Nonetheless, specific emulators may choose to define additional observables or even modify existing ones, so please make sure to *consult your chosen emulator's specific documentation*. 

</div>

### Configuring an `Observable`

After choosing one or more observables, you must then configure them and **provide them to the** `EmulationConfig` (or the chosen backend's specific `EmulationConfig` subclass).

To do so,
- Follow the observable's docstring to instantiate it with the required arguments (if any).
- Optionally, you may also specify custom `evaluation_times` for a given observable - when not given, the emulator will simply use `EmulationConfig.default_evaluation_times` instead.

As a simple example, imagine that by default you only care about an observable's value at the end of the sequence, but you are interested in knowing the energy of the system at the beginning and halfway points of the
execution too. In this case, you could define your observables as follows:

In [ ]:
from pulser.backend import BitStrings, Energy, EmulationConfig

config = EmulationConfig(
    default_evaluation_times=[
        1.0
    ],  # By default, compute an observable only at the end
    observables=[
        BitStrings(),  # No custom evaluation times -> Will be computed only at the end
        Energy(
            evaluation_times=[0.0, 0.5, 1.0]
        ),  # Will be computed at the beginning, middle and end
    ],
)

<div class="alert alert-info">

Note

`evaluation_times` are given as fractions of a sequence's total duration, so `0.0` corresponds to the beginning, `0.5` to the halfway point and `1.0` to the end of the sequence. 

</div>

### `State`- or `Operator`-dependent observables

While most observables can be defined without any arguments, there are two notable exceptions:

- `Fidelity` requires a `State` instance to know against which state it should compute the fidelity;
- `Expectation` requires an `Operator` instance to know which operator's expectation value to compute.

Furthermore, both the `State` and `Operator` subclasses used **must be compatible with the chosen backend**. To make sure this is the case, follow these steps:

1. Pick your target `EmulatorBackend`
2. Take the preferred `EmulationConfig` class from your chosen backend via `EmulatorBackend.config_type`
3. Take the preferred `State` or `Operator` from the config class via `EmulationConfig.state_type` or `EmulationConfig.operator_type`, respectively.

<div class="alert alert-info">

Tip

The example shown below is purposefully agnostic of the emulator backend, all it takes to change backend is to change the `emu_backend_class` variable. 

</div>

In [ ]:
import pulser_simulation
from pulser.backend import Expectation, Fidelity

# Pick a backend, here we chose QutipBackendV2
emu_backend_class = pulser_simulation.QutipBackendV2

config_class = emu_backend_class.config_type  # In this case, `QutipConfig`
state_class = config_class.state_type  # In this case, `QutipState`
operator_class = config_class.operator_type  # In this case, `QutipOperator`


# Arbitrarily chosen fidelity state |rr>
r_state = state_class.from_state_amplitudes(
    eigenstates=("r", "g"),
    amplitudes={"rr": 1.0},
)
# Use `tag_suffix` to better identify the observable in the Results
fidelity = Fidelity(r_state, tag_suffix="rr")

# Arbitrarily chosen operator XX (where X = |r><g|+|g><r|)
pauli_x = operator_class.from_operator_repr(
    eigenstates=("r", "g"),
    n_qudits=2,
    operations=[(1.0, [({"rg": 1.0, "gr": 1.0}, {0, 1})])],
)
# Here we ask for the expectation value at multiple evaluation times
expectation = Expectation(
    pauli_x,
    evaluation_times=[0.0, 0.25, 0.5, 0.75, 1.0],
    # Use `tag_suffix` to better identify the observable in the Results
    tag_suffix="XX",
)

# Creating a new config with the defined observables

config = config_class(
    observables=[fidelity, expectation],
)

## Accessing `Results`

<details>

```python
import numpy as np

import pulser
import pulser_simulation
from pulser.backend import (
    BitStrings,
    Energy,
    Expectation,
    Fidelity,
    StateResult,
)

# STEP 0: Make an arbitrary Pulser Sequence
reg = pulser.Register({"q0": (-5, 0), "q1": (5, 0)})

seq = pulser.Sequence(reg, pulser.AnalogDevice)
seq.declare_channel("rydberg_global", "rydberg_global")

t = 2000  # ns
amp_wf = pulser.BlackmanWaveform(duration=t, area=np.pi)
det_wf = pulser.RampWaveform(duration=t, start=-5, stop=5)
seq.add(pulser.Pulse(amp_wf, det_wf, 0), "rydberg_global")


# STEP 1: Pick the backend and extract the needed classes
emu_backend_class = pulser_simulation.QutipBackendV2

config_class = emu_backend_class.config_type  # In this case, `QutipConfig`
state_class = config_class.state_type  # In this case, `QutipState`
operator_class = config_class.operator_type  # In this case, `QutipOperator`


# STEP 2: Define the desired observables

# Takes `config.default_num_shots` at the `config.default_evaluation_times`
bitstrings = BitStrings()

# Records the state of the systems at the `config.default_evaluation_times`
state_obs = StateResult()

# Records the energy of the system at the beginning, middle and end
energy = Energy(evaluation_times=[0.0, 0.5, 1.0])

# Records fidelity with |rr> at the `config.default_evaluation_times`
r_state = state_class.from_state_amplitudes(
    eigenstates=("r", "g"),
    amplitudes={"rr": 1.0},
)
fidelity = Fidelity(r_state, tag_suffix="rr")

# Records expectation value of XX at custom evaluation times
pauli_x = operator_class.from_operator_repr(
    eigenstates=("r", "g"),
    n_qudits=2,
    operations=[(1.0, [({"rg": 1.0, "gr": 1.0}, {0, 1})])],
)
expectation = Expectation(
    pauli_x, evaluation_times=[0.0, 0.25, 0.5, 0.75, 1.0], tag_suffix="XX"
)

# STEP 3: Creating a new config with the defined observables

config = config_class(
    observables=[bitstrings, state_obs, energy, fidelity, expectation],
    default_evaluation_times=[
        1.0
    ],  # By default, compute an observable only at the end
)

# STEP 4: Run the emulation to get the `Results`

emu_backend = emu_backend_class(seq, config=config)
results = emu_backend.run()
```

</details>

<div class="alert alert-info">

Info

The toggled "Details" above hides a self-contained script replicating many of the steps outlined in the previous section or in the [backend execution page](tutorials/backends.nblink#1.-Creating-the-Pulse-Sequence). 

Feel free to [skip ahead](#Printing-to-get-an-overview-of-the-Results) if you wish to jump straight into how to access `Results`.

</div>

In [ ]:
# NOTE: This should be a hidden cell. If it appears in the built docs,
# make sure `"nbsphinx": "hidden"` is in the cell's metadata.

# -----------------------------------------------------------------------------------
# MAKE SURE THE CODE IN THIS CELL IS THE SAME AS IN THE TOGGLED MARKDOWN CELL ABOVE
# -----------------------------------------------------------------------------------

import numpy as np

import pulser
import pulser_simulation
from pulser.backend import (
    BitStrings,
    Energy,
    Expectation,
    Fidelity,
    StateResult,
)

# STEP 0: Make an arbitrary Pulser Sequence
reg = pulser.Register({"q0": (-5, 0), "q1": (5, 0)})

seq = pulser.Sequence(reg, pulser.AnalogDevice)
seq.declare_channel("rydberg_global", "rydberg_global")

t = 2000  # ns
amp_wf = pulser.BlackmanWaveform(duration=t, area=np.pi)
det_wf = pulser.RampWaveform(duration=t, start=-5, stop=5)
seq.add(pulser.Pulse(amp_wf, det_wf, 0), "rydberg_global")


# STEP 1: Pick the backend and extract the needed classes
emu_backend_class = pulser_simulation.QutipBackendV2

config_class = emu_backend_class.config_type  # In this case, `QutipConfig`
state_class = config_class.state_type  # In this case, `QutipState`
operator_class = config_class.operator_type  # In this case, `QutipOperator`


# STEP 2: Define the desired observables

# Takes `config.default_num_shots` at the `config.default_evaluation_times`
bitstrings = BitStrings()

# Records the state of the systems at the `config.default_evaluation_times`
state_obs = StateResult()

# Records the energy of the system at the beginning, middle and end
energy = Energy(evaluation_times=[0.0, 0.5, 1.0])

# Records fidelity with |rr> at the `config.default_evaluation_times`
r_state = state_class.from_state_amplitudes(
    eigenstates=("r", "g"),
    amplitudes={"rr": 1.0},
)
fidelity = Fidelity(r_state, tag_suffix="rr")

# Records expectation value of XX at custom evaluation times
pauli_x = operator_class.from_operator_repr(
    eigenstates=("r", "g"),
    n_qudits=2,
    operations=[(1.0, [({"rg": 1.0, "gr": 1.0}, {0, 1})])],
)
expectation = Expectation(
    pauli_x, evaluation_times=[0.0, 0.25, 0.5, 0.75, 1.0], tag_suffix="XX"
)

# STEP 3: Creating a new config with the defined observables

config = config_class(
    observables=[bitstrings, state_obs, energy, fidelity, expectation],
    default_evaluation_times=[
        1.0
    ],  # By default, compute an observable only at the end
)

# STEP 4: Run the emulation to get the `Results`

emu_backend = emu_backend_class(seq, config=config)
results = emu_backend.run()

# -----------------------------------------------------------------------------------
# MAKE SURE THE CODE IN THIS CELL IS THE SAME AS IN THE TOGGLED MARKDOWN CELL ABOVE
# -----------------------------------------------------------------------------------

### Printing to get an overview of the `Results`

Given a `Results` instance, the first thing we can do is print it to get an overview of what it contains

In [ ]:
print(results)

As expected, it contains results for each observable we defined, at the requested evaluation times. Now, there are multiple ways we can access these results.

### Getting a list of results for each observable

`Results.get_tagged_results()` returns a dictionary with a list of values for each observable, containing one value per evaluation time. 

In [ ]:
results.get_tagged_results()

To get the list of values for a particular observable, we can either access the dictionary directly or use a shortcut:

In [ ]:
# These are equivalent
results.get_tagged_results()["fidelity_rr"]
results.fidelity_rr

### Getting the evaluation times of an observable

In [ ]:
# These are all equivalent
obs = expectation
results.get_result_times(obs)
results.get_result_times(obs.tag)
results.get_result_times("expectation_XX")

### Getting the result of an observable at a specific evaluation time

In [ ]:
results.get_result("energy", 0.5)

### Shortcuts for final bitstrings and state

Since backend runs typically include the `BitStrings` observable at the end of the sequence, there is a dedicated shortcut to access it.

In [ ]:
# These are equivalent
results.get_result("bitstrings", 1.0)
results.final_bitstrings

The same goes for the quantum state at the end of the emulation

In [ ]:
# These are equivalent
results.get_result("state", 1.0)
results.final_state